In [1]:
# Imports
from time import time

import pyspark.sql.dataframe
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder\
    .config('spark.driver.extraClassPath', '../jar/*')\
    .getOrCreate()

23/02/14 00:55:24 WARN Utils: Your hostname, MacBook-Pro-de-Wylder.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
23/02/14 00:55:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/14 00:55:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def time_measure(function):
    def measure_function(*args, **kwargs):
        start = time()
        c = function(*args, **kwargs)
        print(time() - start)
        return c
    return measure_function

In [3]:
@time_measure
def read_sql_spark(url: str, query: str, spark_session: pyspark.sql.session.SparkSession, is_table:bool=False ) -> pyspark.sql.dataframe.DataFrame:

    df = spark_session.read\
        .format('com.microsoft.sqlserver.jdbc.spark')\
        .option('url', url)\
        .option('dbtable' if is_table else 'query', query).load()

    return df

In [4]:
url_db = 'jdbc:sqlserver://xmdevpronosticos.database.windows.net:1433;database=PronDDAdev;user=admindevpron;password=Pronosticos2019.*'
atypical_date_table_name = '[ETLStaging].[AtypicalDate]'
df_atypical_date = read_sql_spark(url_db, atypical_date_table_name, spark, is_table=True )

df_atypical_date.show()

3.2810611724853516


+------------+----------------+--------------------+-------+-------------------+
|AtypicalDate|AtypicalDateCode|    AtypicalDateName|TypeDay|     XMCreationDate|
+------------+----------------+--------------------+-------+-------------------+
|  2000-01-01|          1--ENE|    Primero de enero|   FEST|2020-02-04 18:31:22|
|  2000-01-02|          2--ENE|        Dos de enero|    ORD|2020-02-04 18:31:22|
|  2000-01-03|           LUNES|Lunes en vacacion...|    ORD|2020-02-04 18:31:22|
|  2000-01-04|          MARTES|Martes en vacacio...|    ORD|2020-02-04 18:31:22|
|  2000-01-05|       MIERCOLES|MiÃ©rcoles en vac...|    ORD|2020-02-04 18:31:22|
|  2000-01-06|          JUEVES|Jueves en vacacio...|    ORD|2020-02-04 18:31:22|
|  2000-01-07|         VIERNES|Viernes en vacaci...|    ORD|2020-02-04 18:31:22|
|  2000-01-08|          SABADO|SÃ¡bado antes de ...|    SAB|2020-02-04 18:31:22|
|  2000-01-09|         DOMINGO|Domingo antes de ...|   FEST|2020-02-04 18:31:22|
|  2000-01-10|              

In [5]:
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

PySpark Version :3.2.1
PySpark Version :3.2.1


In [6]:
from datetime import timedelta, datetime
import pyspark.sql.functions as f

time_exec = '2023-01-24 00:00:00'

forecast_start = '2023-01-23 00:00:00'
forecast_start = datetime.strptime(forecast_start, "%Y-%m-%d %H:%M:%S")
forecast_start = forecast_start.replace(second=0, microsecond=0)
forecast_start = str(forecast_start - timedelta(minutes=5))

In [7]:
@time_measure
def dias(df: pyspark.sql.dataframe.DataFrame):

    df.createOrReplaceTempView('df_calendar')

    df = spark.sql(
        'SELECT *, '
        'DAY(AtypicalDate) as day, '
        'MONTH(AtypicalDate) as month,'
        'YEAR(AtypicalDate) as year '
        'FROM df_calendar'
    )

    return df

In [8]:
df_atypical_date = dias(df_atypical_date)

0.259232759475708


In [9]:
df_atypical_date.show()

+------------+----------------+--------------------+-------+-------------------+---+-----+----+
|AtypicalDate|AtypicalDateCode|    AtypicalDateName|TypeDay|     XMCreationDate|day|month|year|
+------------+----------------+--------------------+-------+-------------------+---+-----+----+
|  2000-01-01|          1--ENE|    Primero de enero|   FEST|2020-02-04 18:31:22|  1|    1|2000|
|  2000-01-02|          2--ENE|        Dos de enero|    ORD|2020-02-04 18:31:22|  2|    1|2000|
|  2000-01-03|           LUNES|Lunes en vacacion...|    ORD|2020-02-04 18:31:22|  3|    1|2000|
|  2000-01-04|          MARTES|Martes en vacacio...|    ORD|2020-02-04 18:31:22|  4|    1|2000|
|  2000-01-05|       MIERCOLES|MiÃ©rcoles en vac...|    ORD|2020-02-04 18:31:22|  5|    1|2000|
|  2000-01-06|          JUEVES|Jueves en vacacio...|    ORD|2020-02-04 18:31:22|  6|    1|2000|
|  2000-01-07|         VIERNES|Viernes en vacaci...|    ORD|2020-02-04 18:31:22|  7|    1|2000|
|  2000-01-08|          SABADO|SÃ¡bado a

In [10]:
@time_measure
def month(df):
    df = df.withColumn('day2', f.dayofmonth('AtypicalDate'))\
        .withColumn('month2', f.month('AtypicalDate'))\
        .withColumn('year2', f.year('AtypicalDate'))

    return df

In [11]:
df_atypical_date_month = month(df_atypical_date)

0.08787870407104492


In [12]:
df_atypical_date_month.show()

+------------+----------------+--------------------+-------+-------------------+---+-----+----+----+------+-----+
|AtypicalDate|AtypicalDateCode|    AtypicalDateName|TypeDay|     XMCreationDate|day|month|year|day2|month2|year2|
+------------+----------------+--------------------+-------+-------------------+---+-----+----+----+------+-----+
|  2000-01-01|          1--ENE|    Primero de enero|   FEST|2020-02-04 18:31:22|  1|    1|2000|   1|     1| 2000|
|  2000-01-02|          2--ENE|        Dos de enero|    ORD|2020-02-04 18:31:22|  2|    1|2000|   2|     1| 2000|
|  2000-01-03|           LUNES|Lunes en vacacion...|    ORD|2020-02-04 18:31:22|  3|    1|2000|   3|     1| 2000|
|  2000-01-04|          MARTES|Martes en vacacio...|    ORD|2020-02-04 18:31:22|  4|    1|2000|   4|     1| 2000|
|  2000-01-05|       MIERCOLES|MiÃ©rcoles en vac...|    ORD|2020-02-04 18:31:22|  5|    1|2000|   5|     1| 2000|
|  2000-01-06|          JUEVES|Jueves en vacacio...|    ORD|2020-02-04 18:31:22|  6|    

In [13]:
@time_measure
def rename(df):
    df = df.withColumn('AtypicalDateCode', f.lower('AtypicalDateCode'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '1--ene', 'ene-1'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '2--ene', 'ene_2'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '1--may', 'may_1'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '20--jul', 'jul_20'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '7--ago', 'ago_7'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '8--dic', 'dic_8'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '24--dic', 'dic_24'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '25--dic', 'dic_25'))\
        .withColumn('AtypicalDateCode', f.regexp_replace('AtypicalDateCode', '31--dic', 'dic_31'))

    return df

In [14]:
df_atypical_date_rename = rename(df_atypical_date_month)

0.5321693420410156


In [15]:
df_atypical_date_rename.show()

+------------+----------------+--------------------+-------+-------------------+---+-----+----+----+------+-----+
|AtypicalDate|AtypicalDateCode|    AtypicalDateName|TypeDay|     XMCreationDate|day|month|year|day2|month2|year2|
+------------+----------------+--------------------+-------+-------------------+---+-----+----+----+------+-----+
|  2000-01-01|           ene-1|    Primero de enero|   FEST|2020-02-04 18:31:22|  1|    1|2000|   1|     1| 2000|
|  2000-01-02|           ene_2|        Dos de enero|    ORD|2020-02-04 18:31:22|  2|    1|2000|   2|     1| 2000|
|  2000-01-03|           lunes|Lunes en vacacion...|    ORD|2020-02-04 18:31:22|  3|    1|2000|   3|     1| 2000|
|  2000-01-04|          martes|Martes en vacacio...|    ORD|2020-02-04 18:31:22|  4|    1|2000|   4|     1| 2000|
|  2000-01-05|       miercoles|MiÃ©rcoles en vac...|    ORD|2020-02-04 18:31:22|  5|    1|2000|   5|     1| 2000|
|  2000-01-06|          jueves|Jueves en vacacio...|    ORD|2020-02-04 18:31:22|  6|    

In [16]:
import numpy as np

In [17]:
list_date_code = np.array(df_atypical_date_rename.select('AtypicalDateCode').distinct().collect())

In [18]:
list_date_code

array([['vss'],
       ['jueves'],
       ['may_1'],
       ['viernes'],
       ['dss'],
       ['dic_25'],
       ['lunes'],
       ['martes'],
       ['dic_31'],
       ['jul_20'],
       ['sss'],
       ['sabado'],
       ['ene_2'],
       ['domingo'],
       ['ene-1'],
       ['miercoles'],
       ['jss'],
       ['lf'],
       ['ago_7'],
       ['mss'],
       ['dic_8'],
       ['dic_24']], dtype='<U9')

In [19]:
list_date_code_2 = [
    'ene_1', 'ene_2', 'may_1', 'jul_20', 'ago_7', 'dic_8', 'dic_24', 'dic_25',
    'dic_31', 'mss', 'jss', 'vss', 'sss', 'dss', 'lf'
]

list_type_day = np.array(df_atypical_date_rename.select('TypeDay').distinct().collect())
list_type_day

array([['SAB'],
       ['FEST'],
       ['ORD']], dtype='<U4')

In [20]:
print(datetime.now())

2023-02-14 00:55:46.351704


In [21]:
variable_type = 'P'
measuring_element_id = 32
date_now = datetime.now().date()

query_input_measure = f'''
    SELECT * FROM [Forecast].[InputMeasure]
    WHERE [MeasuringElementMRID] = {measuring_element_id}
    AND [MeasuresTypeMRID] = '{variable_type}'
    AND [MeasuringDate] BETWEEN DATEADD(DAY,-123,'{date_now}') AND '{date_now}'
'''

input_measure = read_sql_spark(url_db, query_input_measure, spark)

0.8421497344970703


In [22]:
input_measure.show(10000)

+----------------+-------------------+--------------------+----------------+---------------+--------------------+----------+--------------+--------------------+
|InputMeasureMRID|      MeasuringDate|MeasuringElementMRID|MeasuresTypeMRID|ProcessTypeMRID|OriginTimeSeriesMRID|Resolution|MeasuringValue|      XMCreationDate|
+----------------+-------------------+--------------------+----------------+---------------+--------------------+----------+--------------+--------------------+
|      2012131573|2022-10-14 00:00:00|                  32|               P|             20|                  95|         5|       19.5929|2022-10-14 07:06:...|
|      2012123517|2022-10-14 00:05:00|                  32|               P|             20|                  95|         5|       19.4979|2022-10-14 07:06:...|
|      2012127789|2022-10-14 00:10:00|                  32|               P|             20|                  95|         5|       19.2128|2022-10-14 07:06:...|
|      2012130686|2022-10-14 00:15

In [23]:
@time_measure
def query_table_validations():
    dbtable = '[Forecast].[InputMeasure]'
    df_input_measure = read_sql_spark(url_db, dbtable, spark, is_table=True)
    df_input_measure = df_input_measure.filter((df_input_measure.MeasuringElementMRID == measuring_element_id) & (df_input_measure.MeasuresTypeMRID == variable_type))
    return df_input_measure


spark_validation = query_table_validations()

0.7809269428253174
0.884648323059082


In [24]:
spark_validation.show()

+----------------+-------------------+--------------------+----------------+---------------+--------------------+----------+--------------+--------------------+
|InputMeasureMRID|      MeasuringDate|MeasuringElementMRID|MeasuresTypeMRID|ProcessTypeMRID|OriginTimeSeriesMRID|Resolution|MeasuringValue|      XMCreationDate|
+----------------+-------------------+--------------------+----------------+---------------+--------------------+----------+--------------+--------------------+
|      1581098066|2022-01-01 00:00:00|                  32|               P|             20|                  95|         5|       15.5550|2022-03-06 20:06:...|
|      1581098068|2022-01-01 00:05:00|                  32|               P|             20|                  95|         5|       15.4847|2022-03-06 20:06:...|
|      1581071927|2022-01-01 00:10:00|                  32|               P|             20|                  95|         5|       15.4249|2022-03-06 20:06:...|
|      1581085391|2022-01-01 00:15